# L3: Topic Models
### 732A92/TDDE16 Text Mining
Måns Magnusson

The purpose of this lab is to implement the standard Gibbs sampling algorithm for Latent Dirichlet Allocation in Python. You will be supplied starter code, a smaller corpus with State of the Union addresses for the period 1975 to 2000 by paragraph and a list with English stop words. The code is implemented as a class, `LDAGibbs`, where you are expected to replace central parts of the code with your own implementations.

### 1. Sampling

Implement the basic collapsed Gibbs sampling algorithm for Latent Dirichlet Allocation. Use the starter code and add the components that is missing (the sampler part). We use the fact that 

$$p(z_{i}=k)\propto\left(\alpha+n_{d,k}^{(d)}\right)\frac{\left(\beta+n_{k,w_{i}}^{(w)}\right)}{\sum^{V}\left(\beta+n_{k,w_{i}}^{(w)}\right)}=\left(\alpha+n_{d,k}^{(d)}\right)\frac{\left(\beta+n_{k,w_{i}}^{(w)}\right)}{V\beta+n_{k}}$$

to simplify computations, where $K$ is the number of topics, $V$ is the vocabulary size and $D$ is the number of documents. $\mathbf{n}^{(d)}$ is a count matrix of size $D\times K$ with the number of topic indicators by document, $d$, and topic $k$, $\mathbf{n}^{(w)}$ is a count matrix of size $K\times V$ with the number of topic indicators by topic, $k$, and word type, $w$. $\mathbf{n}$ is a topic indicator count vector of length $K$ that contain the number of topic indicators in each topic. The detailed algorithm can be found below:

__Data:__ tokenized corpus $\mathbf{w}$, priors $\alpha, \beta$ <br>
__Result:__ topic indicators $\mathbf{z}$

Init topic indicators $\mathbf{z}$ randomly per token<br>
Init topic probability vector $\mathbf{p}$<br>
Init $\mathbf{n}^{w}$, the topic type count matrix of size ($K \times V$) with respect to $\mathbf{z}$<br>
Init $\mathbf{n}^{d}$, the document topic count matrix of size ($D \times K$) with respect to $\mathbf{z}$<br>
Init $\mathbf{n}$, the topic count vector of length ($K$) with respect to $\mathbf{z}$<br>

for $g \leftarrow 1$ __to__ _num_\__iterations_ __do__<br>
&emsp;&emsp;// Iterate over all tokens<br>
&emsp;&emsp;for $i \leftarrow 1$ __to__ $N$ __do__<br>
&emsp;&emsp;&emsp;&emsp;// Remove current topic indicator $z_i$ from $\mathbf{n}^{w}$, $\mathbf{n}^{d}$ and $\mathbf{n}$<br>
&emsp;&emsp;&emsp;&emsp;$n^{(w)}_{z_i,w_i}$ -= 1, $n^{(d)}_{d_i,z_i}$ -= 1, $n_{z_i}$ -= 1<br>
&emsp;&emsp;&emsp;&emsp;for $k \leftarrow 1$ __to__ $K$ __do__<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;// Compute the unnormalized probability of each topic indicator<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\mathbf{p}_k \leftarrow \left(\alpha+n_{d,k}^{(d)}\right)\frac{\left(\beta+n_{k,w_{i}}^{(w)}\right)}{\left(V\beta+n_{k}\right)}$<br>
&emsp;&emsp;&emsp;&emsp;__end__<br>
&emsp;&emsp;&emsp;&emsp;// Sample the topic indicator<br>
&emsp;&emsp;&emsp;&emsp;$z_i \leftarrow $ Categorical($\mathbf{p}$)<br>
&emsp;&emsp;&emsp;&emsp;// Add the new topic indicator $z_i$ to $\mathbf{n}^{w}$, $\mathbf{n}^{d}$ and $\mathbf{n}$<br>
&emsp;&emsp;&emsp;&emsp;$n^{(w)}_{z_i,w_i}$ += 1, $n^{(d)}_{d_i,z_i}$ += 1, $n_{z_i}$ += 1<br>
&emsp;&emsp;__end__<br>
__end__

For a complete derivation of the collapsed Gibbs sampler for LDA, see https://lingpipe.files.wordpress.com/2010/07/lda3.pdf.

In [19]:
import numpy, random, scipy.special
from tm3 import LDAGibbs
import matplotlib.pyplot as plt

class MyGibbs(LDAGibbs):
        
    def __init__(self, num_topics, docs_file_name, stop_list_file_name = None):
        self.num_topics = num_topics
        self.num_docs = 0
        self.docs = []
        ## Prepare set of stop words
        self.stop_words = set()
        if stop_list_file_name != None:
            with open(stop_list_file_name) as f:
                for line in f:
                    word = line.rstrip()
                    self.stop_words.add(word)
        self.read_documents(docs_file_name)
        self.initialize_matrices()
        self.total_tokens = sum(self.doc_totals)
            
    def read_documents(self, filename):
        """Reads documents from a file, filters stop words and initializes
        the vocabulary. Also converts tokens to integer term IDs."""
        self.vocab = []
        self.vocab_ids = {}
        with open(filename) as f:
            for line in f:
                line = line.replace(".", " ").replace(",", " ").lower()
                self.num_docs += 1
                tokens = []
                for w in line.split():
                    if not w in self.stop_words:
                        if w in self.vocab_ids:
                            tokens.append(self.vocab_ids[w])
                        else:
                            term_id = len(self.vocab)
                            self.vocab.append(w)
                            self.vocab_ids[w] = term_id
                            tokens.append(term_id)
                self.docs.append({ 'tokens': tokens })
        self.num_terms = len(self.vocab)
        print("Read {} documents with a total of {} terms".format(self.num_docs, self.num_terms))
        
    def initialize_matrices(self):
        """Initializes numpy arrays for the matrix computations performed
        by the sampler during the MCMC process."""
        ## Set up numpy matrices
        self.term_topics = numpy.zeros((self.num_terms, self.num_topics)) # n^w
        self.doc_topics = numpy.zeros((self.num_docs, self.num_topics)) # n^d
        self.topic_totals = numpy.zeros(self.num_topics) # n
        self.doc_totals = numpy.zeros(self.num_docs)
        ## Initialize topics randomly
        for doc_id in range(self.num_docs):
            doc = self.docs[doc_id]
            ## Create an array of random topic assignments
            doc['topics'] = [random.randrange(self.num_topics) for token in doc['tokens']]
            ## Construct the initial summary statistics
            doc_length = len(doc['tokens'])
            for token, topic in zip(doc['tokens'], doc['topics']):
                self.term_topics[token][topic] += 1 # n_wk
                self.doc_topics[doc_id][topic] += 1 # n_dk
                self.topic_totals[topic] += 1       # n_k
                self.doc_totals[doc_id] += 1
        ## Printout to check that everything is coherent
        #print(sum(sum(self.doc_topics)))
        #print(sum(sum(self.term_topics)))
        #print(sum(self.topic_totals))
        #print(sum(self.doc_totals))
        
    def run(self, num_iterations = 50, alpha = 0.1, beta = 0.01):
        self.logprobs = []
        for iteration in range(num_iterations): #iteration = 0
            self.make_draw(alpha, beta)
            logprob = self.compute_logprob(alpha, beta)
            self.logprobs.append(logprob)
            print("iteration {}, {}".format(iteration, logprob))
            
    def make_draw(self, alpha, beta):
        ## TODO: implement this function for exercise 1
    
        for doc_id in range(self.num_docs):
            doc = self.docs[doc_id]
            prob_k = numpy.zeros(self.num_topics)
            V = len(self.vocab)
            for token, topic in zip(doc['tokens'], doc['topics']):
                self.term_topics[token][topic] -= 1 # n_wk
                self.doc_topics[doc_id][topic] -= 1 # n_dk
                self.topic_totals[topic]-= 1       # n_k
                ndk = max(0,self.doc_topics[doc_id][topic])
                nkw = max(0,self.term_topics[token][topic])
                nk = max(0,self.topic_totals[topic])
                #for k in range(self.num_topics):
                prob_k[topic] = (alpha + ndk)*(beta + nkw)/(V*beta + nk)
            #print("num topics: ",self.num_topics)
            #print("probs: ", prob_k)            
            num = int(self.num_topics) 
            num = range(num)
            prob_k = prob_k / sum(prob_k)
            print("prob_k ", prob_k)
            #print(len(num))
            z = int(numpy.random.choice(a=num,size=1,p=prob_k))
            self.term_topics[token][z] += 1 # n_wk
            self.doc_topics[doc_id][z] += 1 # n_dk
            self.topic_totals[z] += 1  
            #print("doc.topics", doc['topics'])
            #print("token", token)
            #print("doctopics: ", doc['topics'])
            #print("doctokens: ", doc['tokens'].index(token))
            #print("z:", z)
            doc['topics'][doc['tokens'].index(token)] = z
            self.docs[doc_id] = doc
        return super().make_draw(alpha, beta)
               
            
            
            
#         prob_k = numpy.zeros(self.num_topics)
#         V = len(self.vocab)
#         for token,doc_id in zip(range(V),range(self.num_docs)):
#             print(token,doc_id)
#             for topic in range(self.num_topics):
#             # Remove z_i
#                 self.term_topics[token][topic] -= 1 # n_wk
#                 self.doc_topics[doc_id][topic] -= 1 # n_dk
#                 self.topic_totals[topic] -= 1       # n_k
#                 ndk = max(0,self.doc_topics[doc_id][topic])
#                 nkw = max(0,self.term_topics[token][topic])
#                 nk = max(0,self.topic_totals[topic])
#                 prob_k[topic] = (alpha + ndk)*(beta + nkw)/(V*beta + nk)
#             #print("num topics: ",self.num_topics)
#             #print("probs: ", prob_k)            
#             num = int(self.num_topics) 
#             num = range(num)
#             prob_k = prob_k / sum(prob_k)
#             print("prob_k ", sum(prob_k))
#             #print(len(num))
#             z = int(numpy.random.choice(a=num,size=1,p=prob_k))
#             print("Z:", z)
#             self.term_topics[token][z] += 1 # n_wk
#             self.doc_topics[doc_id][z] += 1 # n_dk
#             self.topic_totals[z] += 1       # n_k
#             doc = self.docs[doc_id]
#             print("doc.topics", doc['topics'])
#             doc['topics'][token] = z
#             self.docs[doc_id] = doc
#             print("After ", doc['topics'])
#             #self.docs['topics'][token]=z
                
        
        
            
    def print_topics(self, j):
        ## TODO: implement this function for exercise 2
        super().print_topics(j)
    
    def plot(self):
        ## TODO: implement this function for exercise 3
        super().plot()            
    
    def compute_logprob(self, alpha, beta):
        ## TODO: implement this function for the bonus exercise
        return super().compute_logprob(alpha, beta)

Implement the `make_draw` function above. You should get behavior very similar to the results from calling the parent class.

In [20]:
num_topics = 10
num_iterations = 10

model = MyGibbs(num_topics, 'sotu_1975_2000.txt', 'stoplist_en.txt')
model.run(num_iterations)

Read 2898 documents with a total of 8695 terms
prob_k  [0.00000000e+00 1.24773059e-01 1.87810190e-01 0.00000000e+00
 2.81025506e-02 0.00000000e+00 6.18549262e-01 1.37865320e-04
 2.68902756e-02 1.37367976e-02]
prob_k  [1.34933549e-04 8.52001972e-01 1.30891733e-04 4.00442917e-02
 1.37862429e-02 9.35040403e-02 1.34164958e-04 0.00000000e+00
 1.30626159e-04 1.32836218e-04]
prob_k  [4.21721971e-02 0.00000000e+00 2.03553936e-04 2.69166268e-01
 2.14366470e-02 2.90656494e-01 2.08645648e-04 3.35112784e-01
 4.08368303e-02 2.06578699e-04]
prob_k  [6.44936931e-03 6.39534911e-05 6.25692785e-03 3.02341724e-01
 6.46047901e-05 6.95076887e-02 2.09613643e-01 2.86650578e-01
 3.09696331e-02 8.80818784e-02]
prob_k  [1.86409963e-01 0.00000000e+00 0.00000000e+00 3.36864384e-01
 1.57012759e-04 1.54965751e-02 2.00807940e-01 1.56348998e-02
 0.00000000e+00 2.44629225e-01]
prob_k  [0.48603103 0.         0.         0.         0.         0.
 0.         0.         0.         0.51396897]
prob_k  [7.36308043e-04 0.0000

/home/732A92/labs/environment/lib/python3.4/site-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in true_divide
/home/732A92/labs/environment/lib/python3.4/site-packages/ipykernel_launcher.py:102: RuntimeWarning: invalid value encountered in less


ValueError: 19 is not in list

### 2. Top terms
Implement the `print_topics` function to extract the top `j` largest counts in $n(w)$ by row. This is the most probable word types in each topic.

In [ ]:
model.print_topics(10)

### 3. Explore the data

Run your implemention on the State of the Union corpus until convergence with 10 topics, don't forget to remove stop words. Plot the log marginal posterior by the number of iterations. How many iterations do you need until convergence? How do you interpret the topics?

[Hint: You can use the plot-function to print the marginal probability for each iteration. To get it working in Jupyter you need to run the command `%matplotlib inline` before plotting the first time.]

In [ ]:
%matplotlib inline
model.plot()

#### Answer here:

### 4. Simulate a new State of the Union speech

Write a function `new_speech` using the `MyGibbs` class to use the estimated values for $\Phi$ from your model (with stop words removed) to simulate a new State of the Union speech. Start out by simulating $\theta_d \sim Dir(\alpha = 0.5)$ and then simulate your document. Does it make sense? Why, why not?

In [ ]:
def new_speech(model, alpha, num_words):
    speech = []        
    return " ".join(speech)

In [ ]:
new_speech(model, 0.5, 100)

#### Answer here:

### Bonus assignment:

To get better understanding how to implement the underlying model or similar models, you might want to implement your own function to compute the log marginal posterior. If so, implement the `compute_logprob` function.

$$\begin{align}
\log p(\mathbf{z}|\mathbf{w}) =& \log\prod^{K}p(\mathbf{w}|\mathbf{z},\beta)\prod^{D}p(\mathbf{z}|\alpha) \\
=& \sum^{K}\log\left[\frac{\Gamma\left(\sum^{V}\beta\right)}{\prod^{V}\Gamma\left(\beta\right)}\frac{\prod^{V}\Gamma\left(n_{kv}^{(w)}+\beta\right)}{\Gamma(\sum^{V}n_{kv}^{(w)}+\beta)}\right]+\sum^{D}\log\left[\frac{\Gamma\left(\sum^{K}\alpha\right)}{\prod^{K}\Gamma\left(\alpha\right)}\frac{\prod^{K}\Gamma\left(n_{dk}^{(d)}+\alpha\right)}{\Gamma(\sum^{K}n_{dk}^{(d)}+\alpha)}\right] \\
=& K\log\Gamma\left(V\beta\right)-KV\log\Gamma\left(\beta\right)+\sum^{K}\sum^{V}\log\Gamma\left(n_{kv}^{(w)}+\beta\right)-\sum^{K}\log\Gamma(\sum^{V}n_{kv}^{(w)}+\beta)\\
&+ D\log\Gamma\left(K\alpha\right)-DK\log\Gamma\left(\alpha\right)+\sum^{D}\sum^{K}\log\Gamma\left(n_{dk}^{(d)}+\alpha\right)-\sum^{D}\log\Gamma(\sum^{K}n_{dk}^{(d)}+\alpha)
\end{align}$$

In Python, use `scipy.special.gammaln` for $\log\Gamma(x)$ (if you run into problems, you might try `math.lgamma` instead).